# Imports

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import branca.colormap as cm
import shapely
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import folium
import osmnx as ox
import matplotlib.pyplot as plt
from descartes import PolygonPatch
from IPython.display import IFrame
import json

In [ ]:
station_df = pd.read_csv('streamlit/input_data/all_stations_by_hour_weekday.csv')

In [ ]:
station_df = gpd.GeoDataFrame(
    station_df, geometry=gpd.points_from_xy(station_df.coordinatesX, station_df.coordinatesY))
station_df["copy_geometry"] = station_df.geometry


In [ ]:
check_df = station_df.groupby('thingID').agg({"geometry":"first", 
                                   "average_res": "std"}).sort_values('average_res', ascending=False)


In [ ]:
check_df

In [ ]:
gpd.GeoDataFrame(
    check_df, geometry=check_df.geometry).explore()

In [ ]:
import plotly.express as px

day_dict = {}
for i in range(7):
    day_dict[i+1] = calendar.day_name[i]
    
station_df["weekday_name"] = station_df["resultWeekday"].map(day_dict)

    
#64
data = station_df.query('thingID == 38').sort_values(['resultWeekday','resultHour'])[["average_res",
                                                                            "weekday_name", "resultHour"]]
data["resultHour"] = data["resultHour"].astype(str)

categories = list(range(23))


fig = px.line_polar(data, r='average_res', theta='resultHour', color='weekday_name', line_close=True, 
                    title="Bikes at a station in Hamburg outskirts by day and hour", height=680,
                   labels={"weekday_name": "Weekday"})
#fig.update_traces(fill='toself')
fig.show()

In [ ]:
station_df = station_df.set_crs('epsg:4236')

In [ ]:
hamburg_df = gpd.read_file('streamlit/input_data/Stadtteile-Hamburg.geojson')
hamburg_df = hamburg_df[["bezirk_name", "stadtteil_name","SHAPE_Area", "geometry"]]
hamburg_districts = pd.read_csv('streamlit/input_data/Stadtteilprofile2021.csv', sep=";")

combine_dict = {
 "Kleiner Grasbrook und Steinwerder": ["Kleiner Grasbrook", "Steinwerder"],
 "Waltershof und Finkenwerder": ["Waltershof", "Finkenwerder"],
 "Neuland und Gut Moor": ["Neuland", "Gut Moor"],
 "Moorburg und Altenwerder": ["Moorburg", "Altenwerder"]
}

for new_val, old_cols in combine_dict.items():
    row_1 = hamburg_df.loc[hamburg_df["stadtteil_name"] == old_cols[0]]
    row_2 = hamburg_df[hamburg_df["stadtteil_name"] == old_cols[1]]

    bezirk = row_1["bezirk_name"]    
    new_geo = pd.concat([row_1, row_2])["geometry"].unary_union
    new_area = float(row_1["SHAPE_Area"]) + float(row_2["SHAPE_Area"])
    
    hamburg_df.loc[len(hamburg_df.index)] = [bezirk, new_val, new_area, new_geo]


hamburg_with_data = pd.merge(left=hamburg_df, right=hamburg_districts, left_on="stadtteil_name", right_on="Stadtgebiet", how="left")

In [ ]:
drop_list = [y for x  in combine_dict.values() for y in  x] + ["Neuwerk"]

In [ ]:
hamburg_with_data = hamburg_with_data.loc[~hamburg_with_data["stadtteil_name"].isin(drop_list )]
hamburg_with_data["Bevölkerung"] = pd.to_numeric(hamburg_with_data["Bevölkerung"].str.replace(" ", ""))

In [ ]:
joined_df = hamburg_with_data.sjoin(station_df, how="left")
agg_df = joined_df.groupby(['stadtteil_name', 'resultHour', 'resultWeekday']).agg({"average_res": sum, "thingID": "nunique", "Bevölkerung": "max"}).reset_index()
agg_df["average_res_pop"] = agg_df["average_res"]/ agg_df["Bevölkerung"]
combined_df = hamburg_with_data.merge(agg_df, how='left', on="stadtteil_name")

# Creating Station Hour Plots

In [ ]:
import osmnx as ox
station_df["fake_date"] = "1980-01-0" + station_df["resultWeekday"].astype(str) +\
        " " + station_df["resultHour"].astype(str) + ":00:00"


trip_times = range(1, 18, 1)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma_r', start=0.3, return_hex=True)

def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

station_df["color"] = station_df["average_res"].apply(color_mapping_function, iso_colors=iso_colors)

def create_geojson_features(df):
    features = []
    
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['coordinatesX'],row['coordinatesY']]
            },
            'properties': {
                'time': str(pd.to_datetime(row['fake_date'])),
                'style': {'color' : ''},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': row["color"],
                    "weight" :0 ,
                    'fillOpacity': 0.7,
                    #'stroke': 'true',
                    'radius': row['average_res'] -5
                }
            }
        }
        features.append(feature)
    return features
features = create_geojson_features(station_df)


In [ ]:
from folium.plugins import TimestampedGeoJson

m = folium.Map(location=[53.555, 9.9914], zoom_start=11,prefer_canvas=True)
TimestampedGeoJson(features,
                  period = 'PT1H',
                  duration = 'PT1H',
                  transition_time = 100,
                   loop=True,
                  auto_play = True,
                  date_options=f"D -- HH:MM",
                  ).add_to(m)

m

# Graph Calculation

In [ ]:
class DictSmallest(dict):
    def __setitem__(self, key, value):
        if (key not in self) or (key in self and self[key] > value):
            dict.__setitem__(self, key, value)
    def update(self, dict):
        for key, value in dict.items():
            if (key not in self) or (key in self and self[key] > value):
                self[key] =  value
        

In [ ]:
unique_station = station_df.groupby('thingID')[["coordinatesY", "coordinatesX"]].min()
unique_station = unique_station.reset_index(drop=True)

all_sub_station = pd.read_csv('streamlit/input_data/cleaned_stations.csv')

In [ ]:
all_sub_station = all_sub_station.rename({"lat": "coordinatesY", "lon": "coordinatesX"}, axis=1)


# Visualization

In [ ]:
%%time

trip_times = range(1, 51, 1)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def create_coords_dict(poi_df, filename, center_location ="Hamburg, Germany",  dist=5000, dict_export=False):
    G =  ox.graph_from_address(center_location, dist=dist, network_type="walk", simplify=True)


    list_of_poi = []

    for index, row in poi_df.iterrows():
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))

    G = ox.project_graph(G)

    node_distances = DictSmallest()

    for poi in tqdm(list_of_poi):
        tmp_res = nx.shortest_path_length(G, source=poi)
        node_distances.update(tmp_res)
        
    
    def color_mapping_function(val, iso_colors):

        for time, color in zip(trip_times, iso_colors):
            if val < time :
                return color

        return iso_colors[-1]

    coords = {key: {"x": G.nodes[key]["lon"], "y": G.nodes[key]["lat"], "distance": node_distances[key],
                    "color": color_mapping_function(node_distances[key], iso_colors)} for key in list(G.nodes())}
    if dict_export:
        with open(f'coords/{filename}.json', 'w') as outfile:
            json.dump(coords, outfile, indent=4)
    return coords
   
def create_folium_plot(coord_file, file_name, caption):

    with open("coords/" + coord_file, "r") as f:
        coords = json.loads(f.read())


    colormap = cm.LinearColormap(colors=iso_colors)
    colormap = colormap.to_step(index=range(0, 51, 5))
    colormap.caption = caption
   
    import folium
    m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True, 
                   )



    for val in coords.values():
        folium.Circle(
          location=[val["y"],val["x"]],
            radius=50,
          #popup="Test",
        stroke=False,
        fill=True,
        color = val["color"],
        fill_opacity=0.3,
        interactive=True

       ).add_to(m)
    colormap.add_to(m)
    m.save(f'streamlit/pages/maps/{file_name}.html')

In [ ]:
%%time
create_coords_dict(unique_station, "all_bike_15000", dist=15000)

In [ ]:
%%time
caption_bike = "Walking distance in minutes to nearest bike station"
create_folium_plot("bike_10000.json", "hamburg_bike_darker_10000",  caption=caption_bike)

# Analysis

In [ ]:
coords_df = pd.read_json("coords/all_bike_15000.json", orient="index")

In [ ]:
coords_df = gpd.GeoDataFrame(
    coords_df, geometry=gpd.points_from_xy(coords_df.x, coords_df.y))

In [ ]:
coords_df = coords_df.reset_index()

In [ ]:
hamburg_with_data["copy_geometry"] = hamburg_with_data.geometry
raw_joined_df = gpd.sjoin(coords_df, hamburg_with_data,  how="left")


### Total Average Walking Distance

In [ ]:
combined_df = raw_joined_df.loc[~raw_joined_df["stadtteil_name"].isna()]
combined_df["n_nodes_plz"]= combined_df.groupby('stadtteil_name')["geometry"].transform("count")
combined_df["pop_perc"] = combined_df["Bevölkerung"] / combined_df["n_nodes_plz"]
combined_df["sum_pop_distance"] = combined_df["distance"] * combined_df["pop_perc"]
combined_df["sum_pop_distance"].sum() / combined_df["pop_perc"].sum()

In [ ]:
%%time
plot_df = combined_df[["geometry", "sum_pop_distance", "distance"]]
trip_times = range(1, 300, 1)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

plot_df = gpd.GeoDataFrame(plot_df, geometry=plot_df["geometry"])
plot_df = plot_df.set_crs(epsg=4326)
plot_df["color"] = plot_df["sum_pop_distance"].apply(color_mapping_function, iso_colors=iso_colors)

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, round(combined_df.sum_pop_distance.max()), 5))
colormap.caption = "Sum of minutes times population"

m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True, 
               )



for index, val in plot_df.iterrows():
    folium.Circle(
      location=[val["geometry"].y,val["geometry"].x],
        radius=50,
      #popup="Test",
    stroke=False,
    fill=True,
    color = val["color"],
    fill_opacity=0.3,
    interactive=True

   ).add_to(m)
colormap.add_to(m)
m.save(f'streamlit/pages/maps/bike_pop_dist.html')


### Walking Distance per District

In [ ]:
combined_df["Bevölkerungs-dichte"]= pd.to_numeric(combined_df["Bevölkerungs-dichte"].str.replace(' ', ''))


In [ ]:
district_df = combined_df.groupby('stadtteil_name').agg({"distance": "mean", 
                            "Bevölkerungs-dichte": "max", "Bevölkerung": "max", "copy_geometry": "first"}).reset_index().sort_values('distance')


In [ ]:
district_df["rank_distance"] = district_df["distance"].rank()
district_df["rank_density"] = district_df["Bevölkerungs-dichte"].rank(ascending=False)
district_df["rank_diff"] = district_df["rank_distance"] - district_df["rank_density"]
district_df["rank_diff_abs"]  = abs(district_df["rank_diff"])

In [ ]:
pd.options.display.max_rows = 30
 

In [ ]:
district_df["distance_pop"] = district_df["distance"] * district_df["Bevölkerung"]

In [ ]:
district_df.sort_values('rank_diff_abs', ascending=False).head(5)

In [ ]:
plot_df = gpd.GeoDataFrame(district_df, geometry=district_df["copy_geometry"])
plot_df = plot_df.set_crs(epsg=4326)

m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True)
#plot_df.explore(m=m, column="distance_pop", cmap="plasma_r", style_kwds={"fillOpacity": 0.75})

### Average bike per District

In [ ]:
station_total_average = gpd.GeoDataFrame(station_df.groupby('thingID').agg({"average_res": "mean", "geometry": "first"}).reset_index())

In [ ]:
combined_station_df = station_total_average.sjoin(hamburg_with_data, how="left")

In [ ]:
district_bikes_df = combined_station_df.groupby('stadtteil_name').agg({"average_res": "sum", "Bevölkerung": "first"}).reset_index()

In [ ]:
district_bikes_df["bikes_per_person"] = district_bikes_df["average_res"] /  district_bikes_df["Bevölkerung"]
district_bikes_df["persons_per_bike"] =   district_bikes_df["Bevölkerung"] / district_bikes_df["average_res"]

In [ ]:
plot_df = pd.concat([district_bikes_df.sort_values("bikes_per_person", ascending=False)[:5], 
                    district_bikes_df.sort_values("bikes_per_person", ascending=False)[-5:]])

In [ ]:
plot_df

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.barplot(x="bikes_per_person", y="stadtteil_name", data=plot_df, orient='h')

ax.bar_label(ax.containers[0], fmt='%.2f')
ax.set_xlim(right=6)
ax.set_ylabel("District Name")
ax.set_xlabel("Bikes per Inhabitant")
ax.set_title('Top and Bottom 5 Districts for average bikes per inhabitant')

In [ ]:
ax.containers[0]

### Average bike per District and hour

In [ ]:
combined_station_df = station_df.sjoin(hamburg_with_data, how="left")

In [ ]:
district_bikes_df = combined_station_df.groupby(['stadtteil_name', 'resultHour', 'resultWeekday']).agg({"average_res": "sum", "Bevölkerung": "first"}).reset_index()

In [ ]:
district_bikes_df["bikes_per_person"] = district_bikes_df["average_res"] /  district_bikes_df["Bevölkerung"]
district_bikes_df["persons_per_bike"] =   district_bikes_df["Bevölkerung"] / district_bikes_df["average_res"]

In [ ]:
district_bikes_df.groupby('stadtteil_name').agg(Mean=("average_res", np.mean),
                                                Min= ("average_res", np.min),
                                                Max= ("average_res", np.max),
                                                Std= ("average_res", np.std)).sort_values('Std', ascending=False)

In [ ]:
import plotly.express as px


day_dict = {}
for i in range(7):
    day_dict[i+1] = calendar.day_name[i]
    
district_bikes_df["weekday_name"] = district_bikes_df["resultWeekday"].map(day_dict)

data = district_bikes_df.query('stadtteil_name == "Hamburg-Altstadt"').sort_values(['resultWeekday','resultHour'])[["average_res",
                                                                            "weekday_name", "resultHour"]]
data["resultHour"] = data["resultHour"].astype(str)

categories = list(range(23))


fig = px.line_polar(data, r='average_res', theta='resultHour', color='weekday_name', line_close=True, 
                    title="Bikes in the district Hamburg-Altstadt by day and hour", height=680,
                   labels={"weekday_name" : "Weekday"})
#fig.update_traces(fill='toself')
fig.show()

## Optmize Stations

In [ ]:
from shapely.geometry import Point
raw_joined_df = gpd.sjoin(coords_df, hamburg_with_data,  how="left")

In [ ]:
original_df = raw_joined_df.loc[~raw_joined_df["stadtteil_name"].isna()]
original_df["n_nodes_plz"]= original_df.groupby('stadtteil_name')["geometry"].transform("count")
original_df["pop_perc"] = original_df["Bevölkerung"] / original_df["n_nodes_plz"]
original_df["sum_pop_distance"] = original_df["distance"] * original_df["pop_perc"]


In [ ]:
G =  ox.graph_from_address("Hamburg, Germany", dist=10000, network_type="walk", simplify=True)

gdf_nodes = ox.graph_to_gdfs(G, edges=False)  

In [ ]:
new_station = unique_station.copy(deep=True)
new_station.loc[len(new_station)] = original_df.sort_values("sum_pop_distance", ascending=False)[["y", "x"]].iloc[0].values

In [ ]:
def get_distance_dict(list_of_poi, G):

    G_projected= ox.project_graph(G)

    node_distances = DictSmallest()

    for poi in list_of_poi:
        tmp_res = nx.shortest_path_length(G_projected, source=poi)
        node_distances.update(tmp_res)

    coords = {key: {"x": G_projected.nodes[key]["lon"], "y": G_projected.nodes[key]["lat"],
                    "distance": node_distances[key]} for key in list(G_projected.nodes())}
    return coords


In [ ]:
def get_new_distances_df(coords_dict, hamburg_with_data):
    coords_df = pd.DataFrame.from_dict(coords_dict, orient="index")
    coords_df = gpd.GeoDataFrame(coords_df, geometry=gpd.points_from_xy(coords_df.x, coords_df.y))
    raw_joined_df = gpd.sjoin(coords_df, hamburg_with_data,  how="left")
    combined_df = raw_joined_df.loc[~raw_joined_df["stadtteil_name"].isna()].copy(deep=True)
    combined_df["n_nodes_plz"]= combined_df.groupby('stadtteil_name')["geometry"].transform("count")
    combined_df["pop_perc"] = combined_df["Bevölkerung"] / combined_df["n_nodes_plz"]
    combined_df["sum_pop_distance"] = combined_df["distance"] * combined_df["pop_perc"]
    return combined_df

In [ ]:
def get_metrics(combined_df, print_output=False):
    sum_pop_minutes = combined_df['sum_pop_distance'].sum()
    average_pop_minutes = combined_df['sum_pop_distance'].sum() / combined_df['pop_perc'].sum()
    if print_output:
        print(f"Total Pop Minutes: {sum_pop_minutes}")
        print(f"Average Pop Minutes: {average_pop_minutes}")
    return sum_pop_minutes, average_pop_minutes

In [ ]:
unique_station = station_df.groupby('thingID')[["coordinatesY", "coordinatesX"]].min()
unique_station = unique_station.reset_index(drop=True)
all_stations = unique_station.copy(deep=True)

In [ ]:
G =  ox.graph_from_address("Hamburg, Germany", dist=15000, network_type="walk", simplify=True)
G_small = ox.graph_from_address("Hamburg, Germany", dist=1000, network_type="walk", simplify=True)


#### Old Values

In [ ]:
list_of_poi = []

for index, row in all_stations.iterrows():
    list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))
    
new_coords = get_distance_dict(list_of_poi, G)

In [ ]:
old_station_df = get_new_distances_df(new_coords, hamburg_with_data)

In [ ]:
get_metrics(old_station_df, print_output=True)

#### Placing new stations

In [ ]:
def place_new_stations(G, original_df, unique_station, n_new_stations=5):
    working_df = original_df.copy(deep=True)
    new_stations_perf = pd.DataFrame(columns=["y", "x", "sum_pop_minutes", "average_pop_minutes"])
    
    list_of_poi = []

    for index, row in unique_station.iterrows():
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.coordinatesY, X=row.coordinatesX))

    
    for n in tqdm(range(n_new_stations)):
        new_y, new_x = working_df.sort_values("sum_pop_distance", ascending=False)[["y", "x"]].iloc[0].values
        list_of_poi.append(ox.distance.nearest_nodes(G, Y=new_y, X=new_x))
        new_coords = get_distance_dict(list_of_poi, G)
        working_df = get_new_distances_df(new_coords, hamburg_with_data).copy(deep=True)
        sum_pop_minutes, average_pop_minutes = get_metrics(working_df)
        working_df.to_csv('working_df.csv', index=False)        
        print(new_y, new_x, sum_pop_minutes, average_pop_minutes)
        new_stations_perf.loc[len(new_stations_perf)] = [new_y, new_x, sum_pop_minutes, average_pop_minutes]
        new_stations_perf.to_csv('new_stations_perf.csv', index=False)
    return (new_stations_perf, working_df)

In [ ]:
test_perf, working_df = place_new_stations(G, original_df, unique_station, n_new_stations=100)

In [ ]:
test_perf = pd.read_csv('new_stations_perf.csv')
test_perf = gpd.GeoDataFrame(
        test_perf, geometry=gpd.points_from_xy(test_perf.x, test_perf.y))

In [ ]:
test_perf.average_pop_minutes.plot(xlabel="Number of new stations", ylabel="Average Walking Distance")

In [ ]:
working_df = pd.read_csv('working_df.csv')

In [ ]:
%%time
new_coords = get_distance_dict(list_of_poi, G)
working_df = get_new_distances_df(new_coords, hamburg_with_data).copy(deep=True)

In [ ]:
district_df = combined_df.groupby('stadtteil_name').agg({"distance": "mean", 
                            "Bevölkerungs-dichte": "max", "Bevölkerung": "max", "copy_geometry": "first"}).reset_index().sort_values('distance')

district_df["distance_pop"] = district_df["distance"] * district_df["Bevölkerung"]
plot_df = gpd.GeoDataFrame(district_df, geometry=district_df["copy_geometry"])
plot_df = plot_df.set_crs(epsg=4326)

m = folium.Map(location=[53.555, 9.9914], zoom_start=11,prefer_canvas=True)

plot_df.explore(m=m, column="distance_pop", cmap="plasma_r", 
                legend_kwds={'max_labels':6},
                style_kwds={"fillOpacity": 0.75})

test_perf.explore(m=m)

In [ ]:
max_value = round(plot_df["distance_pop"].max())
min_value = round(plot_df["distance_pop"].min())

In [ ]:
%%time
plot_df = working_df[["geometry", "sum_pop_distance", "distance"]]
trip_times = range(1, 700)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

plot_df["geometry"] = plot_df["geometry"].apply(wkt.loads)

plot_df = gpd.GeoDataFrame(plot_df, geometry=plot_df["geometry"])
plot_df = plot_df.set_crs(epsg=4326)
plot_df["color"] = plot_df["sum_pop_distance"].apply(color_mapping_function, iso_colors=iso_colors)

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 700))
colormap.caption = "Weighted walking distance by population"

m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True)

for index, val in plot_df.iterrows():
    folium.Circle(
      location=[val["geometry"].y,val["geometry"].x],
        radius=50,
      #popup="Test",
    stroke=False,
    fill=True,
    color = val["color"],
    fill_opacity=0.3,
    interactive=True

   ).add_to(m)
colormap.add_to(m)
test_perf.explore(m=m, marker_kwds={"radius": 5})
m.save(f'streamlit/pages/maps/new_bike_dist_pop.html')

In [ ]:
%%time
plot_df = combined_df[["geometry", "sum_pop_distance", "distance"]]
trip_times = range(1, 700)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

#plot_df["geometry"] = plot_df["geometry"].apply(wkt.loads)

plot_df = gpd.GeoDataFrame(plot_df, geometry=plot_df["geometry"])
plot_df = plot_df.set_crs(epsg=4326)
plot_df["color"] = plot_df["sum_pop_distance"].apply(color_mapping_function, iso_colors=iso_colors)

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 700))
colormap.caption = "Weighted walking distance by population"

m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True)

for index, val in plot_df.iterrows():
    folium.Circle(
      location=[val["geometry"].y,val["geometry"].x],
        radius=50,
      #popup="Test",
    stroke=False,
    fill=True,
    color = val["color"],
    fill_opacity=0.3,
    interactive=True

   ).add_to(m)
colormap.add_to(m)
m.save(f'streamlit/pages/maps/bike_dist_pop.html')

In [ ]:
combined_df.sum_pop_distance.describe()

In [ ]:
%%time
plot_df = working_Df[["geometry", "sum_pop_distance", "distance"]]
trip_times = range(1, 50, 1)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

plot_df["geometry"] = plot_df["geometry"].apply(wkt.loads)

plot_df = gpd.GeoDataFrame(plot_df, geometry=plot_df["geometry"])
plot_df = plot_df.set_crs(epsg=4326)
plot_df["color"] = plot_df["distance"].apply(color_mapping_function, iso_colors=iso_colors)

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 50, 1))
colormap.caption = "Average walking distance"

m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True, 
               )



for index, val in plot_df.iterrows():
    folium.Circle(
      location=[val["geometry"].y,val["geometry"].x],
        radius=50,
      #popup="Test",
    stroke=False,
    fill=True,
    color = val["color"],
    fill_opacity=0.3,
    interactive=True

   ).add_to(m)
colormap.add_to(m)
test_perf.explore(m=m, marker_kwds={"radius": 5})
m.save(f'streamlit/pages/maps/new_bike_dist.html')


In [ ]:
%%time

trip_times = range(1, 51, 1)
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

for index, row in tqdm(test_perf.iterrows()):
    list_of_poi.append(ox.distance.nearest_nodes(G, Y=row["y"], X=row["x"]))
    

G = ox.project_graph(G)

node_distances = DictSmallest()


In [ ]:
node_distances = DictSmallest()

for poi in tqdm(list_of_poi):
    tmp_res = nx.shortest_path_length(G, source=poi)
    node_distances.update(tmp_res)

In [ ]:
def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

coords = {key: {"x": G.nodes[key]["lon"], "y": G.nodes[key]["lat"], "distance": node_distances[key],
                "color": color_mapping_function(node_distances[key], iso_colors)} for key in list(G.nodes())}


In [ ]:
%%time
colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 51, 5))
colormap.caption = "Walking distance in minutes to nearest bike station"

m = folium.Map(location=[53.555, 9.9914], zoom_start=12,prefer_canvas=True, 
               )



for val in coords.values():
    folium.Circle(
      location=[val["y"],val["x"]],
        radius=50,
      #popup="Test",
    stroke=False,
    fill=True,
    color = val["color"],
    fill_opacity=0.3,
    interactive=True

   ).add_to(m)
colormap.add_to(m)
test_perf.explore(m=m, marker_kwds={"radius": 5})

m.save(f'streamlit/pages/maps/bike_with_new_stations.html')